# Dog Breed Classification

Playing with TensorFlow and Neural Networks to build a classifier to predict the breed given an image of a dog. The source of the dataset was Kaggle and the competition related to this task can be found [here](https://www.kaggle.com/c/dog-breed-identification). Storing the training data on Google Drive for easy integration with colab. Using Colab to take advantage of the compute power and I'm not sure my current laptop could cope with this task.

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
from PIL import Image
print(tf.VERSION)

1.13.1
